Here we calculate the true value under any censoring mechanism, when A=1.

In [ ]:
#  t=3
# calculate the true value
set.seed(1)
n = 1e6
W = rbinom(n=n, size = 1, prob = 0.5)
A = rep(1, n)
L1 = rbinom(n=n, size = 1, prob = plogis(1.5*W + 3*A - W*A - 2))
C1 = rep(0,n)
L2 = rbinom(n=n, size=1, prob = plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
L2_obs = ifelse(C1==1, NA, L2)
C2 = rep(0,n)
Y = rbinom(n=n, size = 1, prob = plogis(1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2))
Y_obs = ifelse(C2==1 , NA, Y)
truth3 = mean(Y_obs)
#this should always be the truth regardless of censoring mechanism
print(truth3)

Install and load important packages

In [ ]:
install.packages("SuperLearner")
install.packages("ltmle")
library(tidyverse)
library(SuperLearner)
library(ltmle)

Implementation of LTMLE package for comparison

In [ ]:
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.1)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.1)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

C1 = 1-C1
C2 = 1-C2
# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)
dat_clean = na.omit(dat)
SL.library <- c("SL.glm.interaction", "SL.mean")
res <- ltmle(
  data   = dat_clean,
  Anodes = "A",
  Lnodes = c("L1", "L2_obs"),   # match column names exactly
  Cnodes = c("C1", "C2"),
  Ynodes = "Y_obs",
  abar   = 1,
  SL.library = SL.library
)

res$estimates["tmle"]
res$estimates["iptw"]

res_gcomp <- ltmle(
  data   = dat_clean,
  Anodes = "A",
  Lnodes = c("L1", "L2_obs"),   # match column names exactly
  Cnodes = c("C1", "C2"),
  Ynodes = "Y_obs",
  abar   = 1,
  SL.library = SL.library,
  gcomp = TRUE
)
res_gcomp


In [ ]:
# ---- helper to run one Monte‑Carlo draw ----
run_once <- function(seed) {
  set.seed(seed)

  n <- 500
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5*W - 1))

  # time‑1 covariates
  L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
  C1 <- rbinom(n, 1, 0.1)

  # time‑2 covariates
  L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
  C2 <- rbinom(n, 1, 0.1)

  # observed values
  L2_obs <- ifelse(C1 == 1, NA, L2)

  # outcome (only if uncensored at both times)
  Y <- rbinom(
    n, 1,
    plogis(1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  # convert to “uncensored = 1” format required by ltmle
  C1 <- 1 - C1
  C2 <- 1 - C2

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)
  dat_clean <- na.omit(dat)             # keep fully observed rows

  SL.lib <- c("SL.glm.interaction", "SL.mean")

  # TMLE + IPTW
  fit_tmle <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib
  )

  # G‑computation (skip targeting step)
  fit_gcomp <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib,
    gcomp = TRUE
  )

  data.frame(
    seed  = seed,
    gcomp = as.numeric(fit_gcomp$estimates["gcomp"]),
    tmle  = as.numeric(fit_tmle$estimates["tmle"]),
    iptw  = as.numeric(fit_tmle$estimates["iptw"])          # IPTW estimate
  )
}

# ---- run simulation ----
seeds <- 2025 + 1:200
sim_results <- do.call(rbind, lapply(seeds, run_once))

print(sim_results, digits = 4)


In [ ]:
# ---- helper to get the three metrics ----
get_metrics <- function(x, truth) {
  bias <- mean(x) - truth
  varx <- var(x)
  mse  <- bias^2 + varx      # equivalently mean( (x - truth)^2 )
  c(bias = bias, variance = varx, mse = mse)
}

metrics <- rbind(
  gcomp = get_metrics(sim_results$gcomp, truth3),
  tmle  = get_metrics(sim_results$tmle,  truth3),
  iptw  = get_metrics(sim_results$iptw,  truth3)
)

print(round(metrics, 6))

The chunks below contains the  code for Ori's longitudinal tmle

In [ ]:
# DGP 1 (the original one), low independent censoring
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.1)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.1)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)


In [ ]:
# Targeting Y
# Set intervention level
abar <- 1

# Define SL library
SL.library <- c("SL.glm.interaction", "SL.mean")

# Fit g_A: P(A = 1 | W)
gA_fit <- SuperLearner(Y = dat$A, X = data.frame(W = dat$W),
                       family = binomial(), SL.library = SL.library)
gA_pred <- predict(gA_fit, newdata = data.frame(W = dat$W), type = "response")$pred

# Fit g_C1: P(C1 = 1 | A, W, L1)
gC1_fit <- SuperLearner(Y = dat$C1, X = dat[, c("A", "W", "L1")],
                        family = binomial(), SL.library = SL.library)
gC1_pred <- predict(gC1_fit, newdata = dat[, c("A", "W", "L1")], type = "response")$pred

# Fit g_C2: P(C2 = 1 | A, W, L1, L2)
gC2_fit <- SuperLearner(Y = dat$C2[!is.na(dat$L2_obs)], X = dat[!is.na(dat$L2_obs), c("A", "W", "L1", "L2_obs")],
                        family = binomial(), SL.library = SL.library)
gC2_pred <- predict(gC2_fit, newdata = dat[, c("A", "W", "L1", "L2_obs")], type = "response")$pred

# Now construct the clever covariate:
# H_Y = I(A==1 & C1==0 & C2==0) / [gA * (1 - gC1) * (1 - gC2)]

clever_covariate_y_g <- (dat$A == 1 & dat$C1 == 0 & dat$C2 == 0) /
                    (gA_pred * (1 - gC1_pred) * (1 - gC2_pred))

# Fit initial Q_Y
qY_fit <- SuperLearner(Y = dat$Y_obs[!is.na(dat$Y_obs)],
                       X = dat[!is.na(dat$Y_obs), c("L2_obs", "L1", "A", "W")],
                       family = binomial(), SL.library = SL.library)

# Predictions for different histories of L(t) and updates
L11 = dat
L11$C1 = 0
L11$C2 = 0
L11$L1 = 1
L11$L2_obs = 1
L11$A = 1
qy.fit.L11 = predict(qY_fit,type = "link",newdata = L11)$pred
qy.fit.L11.update = glm(Y ~ offset(qy.fit.L11)+1, weights = clever_covariate_y_g, family = "quasibinomial", data = L11)
qy.star.L11 = predict(qy.fit.L11.update, type = "response", newdata=dat)

# Update P(Y = 1, C1 = 0, C2 = 0, A = 1 | L2 = 1, L1 = 0, W)
L01 = dat
L01$C1 = 0
L01$C2 = 0
L01$L1 = 1
L01$L2_obs = 0
L01$A = 1
qy.fit.L01 = predict(qY_fit,type = "link",newdata = L01)$pred
qy.fit.L01.update = glm(Y ~ offset(qy.fit.L01)+1, weights = clever_covariate_y_g, family = "quasibinomial", data = L01)
qy.star.L01 = predict(qy.fit.L01.update, type = "response", newdata=dat)

# Update P(Y = 1, C1 = 0, C2 = 0, A = 1 | L2 = 0, L1 = 1, W)
L10 = dat
L10$C1 = 0
L10$C2 = 0
L10$L1 = 0
L10$L2_obs = 1
L10$A = 1
qy.fit.L10 = predict(qY_fit,type = "link",newdata = L10)$pred
qy.fit.L10.update = glm(Y ~ offset(qy.fit.L10)+1, weights = clever_covariate_y_g, family = "quasibinomial", data = L10)
qy.star.L10 = predict(qy.fit.L10.update, type = "response", newdata=dat)

# Update P(Y = 1, C1 = 0, C2 = 0, A = 1 | L2 = 0, L1 = 0, W)
L00 = dat
L00$C1 = 0
L00$C2 = 0
L00$L1 = 0
L00$L2_obs = 0
L00$A = 1
qy.fit.L00 = predict(qY_fit,type = "link",newdata = L00)$pred
qy.fit.L00.update = glm(Y ~ offset(qy.fit.L00)+1, weights = clever_covariate_y_g, family = "quasibinomial", data = L00)
qy.star.L00 = predict(qy.fit.L00.update, type = "response", newdata=dat)

# Targeting L2

# Update P(L2 = 1| C1 = 0, L1 = 1, A1 = 1, W)
Cl2q1 = qy.star.L11 - qy.star.L01
Cl2q0 = qy.star.L10 - qy.star.L00
Cl2g = as.numeric(dat$A == abar & dat$C1 == 0)/(gA_pred*(1-gC1_pred))


qL2_fit <- SuperLearner(Y = dat$L2_obs[!is.na(dat$L2_obs)], X = dat[!is.na(dat$L2_obs), c("A", "W", "L1")],
                        family = binomial(), SL.library = SL.library)
#Q_L2_init <- predict(qL2_fit, newdata = dat[, c("A", "W", "L1")], type = "response")$pred
#logit_Q_L2 <- qlogis(Q_L2_init)

#Make predictions for L1=1 and L0=0
L1=dat
L1$A=abar
L1$C1=0
L1$L1=1
ql2.fit.L1 = predict(qL2_fit,type = "link",newdata = L1)$pred
ql2.fit.L1.update = glm(L2_obs ~ offset(ql2.fit.L1)+Cl2q1, weights = Cl2g, family = "quasibinomial", data = L1)
ql2.star.L1 = predict(ql2.fit.L1.update, type = "response", newdata=dat)


L0=dat
L0$A=abar
L0$C1=0
L0$L1=0
ql2.fit.L0 = predict(qL2_fit,type = "link",newdata = L0)$pred
ql2.fit.L0.update = glm(L2_obs ~ offset(ql2.fit.L0)+Cl2q0, weights = Cl2g, family = "quasibinomial", data = L0)
ql2.star.L0 = predict(ql2.fit.L0.update, type = "response", newdata=dat)

C_Q <- qy.star.L01*(1-ql2.star.L1)+qy.star.L11*ql2.star.L1- (qy.star.L00*(1-ql2.star.L0)+qy.star.L10*ql2.star.L0)
Cl2g = as.numeric(dat$A == abar)/(gA_pred)

qL1_fit <- SuperLearner(Y = dat$L1, X = dat[, c("A", "W")],
                        family = binomial(), SL.library = SL.library)
L <- dat
L$A <- abar

ql1.fit = predict(qL1_fit, type="link", newdata=L)$pred
ql1.fit.update = glm(L1~offset(ql1.fit)+C_Q, weights = Cl2g, family= "quasibinomial", data=dat)
ql1.star = predict(ql1.fit.update, type="response")

ori_estimate=mean(qy.star.L11*ql2.star.L1*ql1.star+qy.star.L10*ql2.star.L0*(1-ql1.star)+qy.star.L01*(1-ql2.star.L1)*ql1.star+ qy.star.L00*(1-ql2.star.L0)*(1-ql1.star))


In [ ]:
ori_estimate

In [ ]:
run_one_targeted_tmle <- function(seed,
                                  n = 500,
                                  SL.library = c("SL.glm.interaction", "SL.mean"),
                                  abar = 1) {
  set.seed(seed)

  ## --- Data‑generating process (DGP 1, low independent censoring) ----
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5 * W - 1))

  L1 <- rbinom(n, 1, plogis(1.5 * W + 3 * A - W * A - 2))
  C1 <- rbinom(n, 1, 0.1)

  L2 <- rbinom(n, 1, plogis(1.5 * W + 2 * A - 2 * W * A + L1 - 1))
  C2 <- rbinom(n, 1, 0.1)
  L2_obs <- ifelse(C1 == 1, NA, L2)

  Y <- rbinom(
    n, 1,
    plogis(1.5 * W + 2 * A - 1.5 * W * A * L1 + 3 * L1 -
             1.5 * L1 * A - 2 - L2 * W + 2 * L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)

  ## --- g‑models -------------------------------------------------------
  gA_fit  <- SuperLearner(Y = dat$A,
                          X = data.frame(W = dat$W),
                          family = binomial(),
                          SL.library = SL.library)
  gA_pred <- predict(gA_fit, newdata = data.frame(W = dat$W),
                     type = "response")$pred

  gC1_fit  <- SuperLearner(Y = dat$C1,
                           X = dat[, c("A", "W", "L1")],
                           family = binomial(),
                           SL.library = SL.library)
  gC1_pred <- predict(gC1_fit,
                      newdata = dat[, c("A", "W", "L1")],
                      type = "response")$pred

  idx_L2 <- !is.na(dat$L2_obs)
  gC2_fit <- SuperLearner(Y = dat$C2[idx_L2],
                          X = dat[idx_L2, c("A", "W", "L1", "L2_obs")],
                          family = binomial(),
                          SL.library = SL.library)
  gC2_pred <- predict(gC2_fit,
                      newdata = dat[, c("A", "W", "L1", "L2_obs")],
                      type = "response")$pred

  ## --- clever covariate for Y ----------------------------------------
  H_Y <- (dat$A == 1 & dat$C1 == 0 & dat$C2 == 0) /
         (gA_pred * (1 - gC1_pred) * (1 - gC2_pred))

  ## --- Q‑model for Y --------------------------------------------------
  idx_Y <- !is.na(dat$Y_obs)
  qY_fit <- SuperLearner(Y = dat$Y_obs[idx_Y],
                         X = dat[idx_Y, c("L2_obs", "L1", "A", "W")],
                         family = binomial(),
                         SL.library = SL.library)

  # helper to create a modified data‑set with specific (L1, L2) pattern
  make_hist <- function(dat0, L1_val, L2_val) {
    h <- dat0
    h$C1      <- 0
    h$C2      <- 0
    h$A       <- 1
    h$L1      <- L1_val
    h$L2_obs  <- L2_val
    h
  }

  # four histories
  hist_L11 <- make_hist(dat, 1, 1)
  hist_L10 <- make_hist(dat, 0, 1)
  hist_L01 <- make_hist(dat, 1, 0)
  hist_L00 <- make_hist(dat, 0, 0)

  update_Y <- function(hist_df) {
    off   <- predict(qY_fit, type = "link", newdata = hist_df)$pred
    eps   <- glm(Y_obs ~ offset(off) + 1,
                 weights = H_Y,
                 family  = quasibinomial(),
                 data    = cbind(hist_df, Y_obs = dat$Y_obs))$coef
    plogis(off + eps)
  }

  qy_star_L11 <- update_Y(hist_L11)
  qy_star_L01 <- update_Y(hist_L01)
  qy_star_L10 <- update_Y(hist_L10)
  qy_star_L00 <- update_Y(hist_L00)

  ## --- Target L2 ------------------------------------------------------
  Cl2q1 <- qy_star_L11 - qy_star_L01
  Cl2q0 <- qy_star_L10 - qy_star_L00
  Cl2g  <- (dat$A == abar & dat$C1 == 0) / (gA_pred * (1 - gC1_pred))

  qL2_fit <- SuperLearner(Y = dat$L2_obs[idx_L2],
                          X = dat[idx_L2, c("A", "W", "L1")],
                          family = binomial(),
                          SL.library = SL.library)

  # L1 = 1
  hist_L1 <- make_hist(dat, 1, dat$L2_obs)
  off1 <- predict(qL2_fit, type = "link", newdata = hist_L1)$pred
  eps1 <- glm(L2_obs ~ offset(off1) + Cl2q1,
              weights = Cl2g,
              family  = quasibinomial(),
              data    = cbind(hist_L1, L2_obs = dat$L2_obs))$coef
  ql2_star_L1 <- plogis(off1 + eps1)

  # L1 = 0
  hist_L0 <- make_hist(dat, 0, dat$L2_obs)
  off0 <- predict(qL2_fit, type = "link", newdata = hist_L0)$pred
  eps0 <- glm(L2_obs ~ offset(off0) + Cl2q0,
              weights = Cl2g,
              family  = quasibinomial(),
              data    = cbind(hist_L0, L2_obs = dat$L2_obs))$coef
  ql2_star_L0 <- plogis(off0 + eps0)

  ## --- Target L1 ------------------------------------------------------
  C_Q  <- qy_star_L01 * (1 - ql2_star_L1) + qy_star_L11 * ql2_star_L1 -
          (qy_star_L00 * (1 - ql2_star_L0) + qy_star_L10 * ql2_star_L0)
  Cl1g <- (dat$A == abar) / gA_pred

  qL1_fit <- SuperLearner(Y = dat$L1,
                          X = dat[, c("A", "W")],
                          family = binomial(),
                          SL.library = SL.library)
  hist_L <- dat
  hist_L$A <- abar
  offL1 <- predict(qL1_fit, type = "link", newdata = hist_L)$pred
  epsL1 <- glm(L1 ~ offset(offL1) + C_Q,
               weights = Cl1g,
               family  = quasibinomial(),
               data    = hist_L)$coef
  ql1_star <- plogis(offL1 + epsL1)

  ## --- Final plug‑in estimate ----------------------------------------
  ori_estimate <- mean(
    qy_star_L11 * ql2_star_L1 * ql1_star +
    qy_star_L10 * ql2_star_L0 * (1 - ql1_star) +
    qy_star_L01 * (1 - ql2_star_L1) * ql1_star +
    qy_star_L00 * (1 - ql2_star_L0) * (1 - ql1_star)
  )

  return(ori_estimate)
}

# ─────────────────────────────────────────────────────────────
# Five repetitions with seeds 2026 … 2030
# ─────────────────────────────────────────────────────────────
seeds <- 2025 + 1:200           # 2026, 2027, 2028, 2029, 2030
ori_out <- sapply(seeds, run_one_targeted_tmle)

sim_results_ori <- data.frame(seed = seeds, ori_estimate = ori_out)
print(sim_results_ori, digits = 5)


In [ ]:
bias      <- mean(sim_results_ori$ori_estimate) - truth3
variance  <- var(sim_results_ori$ori_estimate)
mse       <- bias^2 + variance          # or mean((ori_estimate - truth3)^2)

metrics_ori <- c(bias = bias, variance = variance, mse = mse)
print(round(metrics_ori, 6))

Here are other DGP's I'd want to test. As far as analysis goes, all that changes should just be DGP and the rest of the code is the same.

In [ ]:
# DGP 1 (the original one), low independent censoring
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.1)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.1)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)


# DGP 2 (the original one), high independent censoring
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.5)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.5)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)


# DGP 3 -informative
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, plogis(-3+0.8*A+0.3*W+0.25*L1))

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, plogis(-3+0.8*A+0.3*W+0.25*L1+0.25*L2))
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)


In [ ]:
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.1)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.1)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

In [ ]:
#DGP 2, high independent censoring
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, 0.5)

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, 0.5)
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)


In [ ]:
# ─────────────────────────────────────────────────────────────
# 1.  One run of the high‑censoring DGP  (DGP 2)
# ─────────────────────────────────────────────────────────────
run_one_targeted_tmle_dgp2 <- function(seed,
                                       n = 500,
                                       SL.library = c("SL.glm.interaction", "SL.mean"),
                                       abar = 1) {
  set.seed(seed)

  ## ----------  DGP 2 :  high independent censoring ----------
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5 * W - 1))

  L1 <- rbinom(n, 1, plogis(1.5 * W + 3 * A - W * A - 2))
  C1 <- rbinom(n, 1, 0.5)                         # ← 0.50 now

  L2 <- rbinom(n, 1, plogis(1.5 * W + 2 * A - 2 * W * A + L1 - 1))
  C2 <- rbinom(n, 1, 0.5)                         # ← 0.50 now
  L2_obs <- ifelse(C1 == 1, NA, L2)

  Y <- rbinom(
    n, 1,
    plogis(1.5 * W + 2 * A - 1.5 * W * A * L1 +
           3 * L1 - 1.5 * L1 * A - 2 - L2 * W + 2 * L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)

  ## ----------  identical targeting steps as before ----------
  # (function body identical to DGP 1 after this point)
  # -- g‑models
  gA_fit  <- SuperLearner(Y = dat$A,
                          X = data.frame(W = dat$W),
                          family = binomial(),
                          SL.library = SL.library)
  gA_pred <- predict(gA_fit, newdata = data.frame(W = dat$W),
                     type = "response")$pred

  gC1_fit  <- SuperLearner(Y = dat$C1,
                           X = dat[, c("A", "W", "L1")],
                           family = binomial(),
                           SL.library = SL.library)
  gC1_pred <- predict(gC1_fit,
                      newdata = dat[, c("A", "W", "L1")],
                      type = "response")$pred

  idx_L2   <- !is.na(dat$L2_obs)
  gC2_fit  <- SuperLearner(Y = dat$C2[idx_L2],
                           X = dat[idx_L2, c("A", "W", "L1", "L2_obs")],
                           family = binomial(),
                           SL.library = SL.library)
  gC2_pred <- predict(gC2_fit,
                      newdata = dat[, c("A", "W", "L1", "L2_obs")],
                      type = "response")$pred

  # clever covariate for Y
  H_Y <- (dat$A == 1 & dat$C1 == 0 & dat$C2 == 0) /
         (gA_pred * (1 - gC1_pred) * (1 - gC2_pred))

  # initial Q_Y
  idx_Y <- !is.na(dat$Y_obs)
  qY_fit <- SuperLearner(Y = dat$Y_obs[idx_Y],
                         X = dat[idx_Y, c("L2_obs", "L1", "A", "W")],
                         family = binomial(),
                         SL.library = SL.library)

  make_hist <- function(dat0, L1_val, L2_val) {
    h <- dat0
    h$C1      <- 0
    h$C2      <- 0
    h$A       <- 1
    h$L1      <- L1_val
    h$L2_obs  <- L2_val
    h
  }
  update_Y <- function(hist_df) {
    off <- predict(qY_fit, type = "link", newdata = hist_df)$pred
    eps <- glm(Y_obs ~ offset(off) + 1,
               weights = H_Y,
               family = quasibinomial(),
               data = cbind(hist_df, Y_obs = dat$Y_obs))$coef
    plogis(off + eps)
  }

  qy_star_L11 <- update_Y(make_hist(dat, 1, 1))
  qy_star_L10 <- update_Y(make_hist(dat, 0, 1))
  qy_star_L01 <- update_Y(make_hist(dat, 1, 0))
  qy_star_L00 <- update_Y(make_hist(dat, 0, 0))

  # ----- Target L2
  Cl2q1 <- qy_star_L11 - qy_star_L01
  Cl2q0 <- qy_star_L10 - qy_star_L00
  Cl2g  <- (dat$A == abar & dat$C1 == 0) / (gA_pred * (1 - gC1_pred))

  qL2_fit <- SuperLearner(Y = dat$L2_obs[idx_L2],
                          X = dat[idx_L2, c("A", "W", "L1")],
                          family = binomial(),
                          SL.library = SL.library)

  hist_L1 <- make_hist(dat, 1, dat$L2_obs)
  off1 <- predict(qL2_fit, type = "link", newdata = hist_L1)$pred
  eps1 <- glm(L2_obs ~ offset(off1) + Cl2q1,
              weights = Cl2g,
              family = quasibinomial(),
              data = cbind(hist_L1, L2_obs = dat$L2_obs))$coef
  ql2_star_L1 <- plogis(off1 + eps1)

  hist_L0 <- make_hist(dat, 0, dat$L2_obs)
  off0 <- predict(qL2_fit, type = "link", newdata = hist_L0)$pred
  eps0 <- glm(L2_obs ~ offset(off0) + Cl2q0,
              weights = Cl2g,
              family = quasibinomial(),
              data = cbind(hist_L0, L2_obs = dat$L2_obs))$coef
  ql2_star_L0 <- plogis(off0 + eps0)

  # ----- Target L1
  C_Q  <- qy_star_L01 * (1 - ql2_star_L1) + qy_star_L11 * ql2_star_L1 -
          (qy_star_L00 * (1 - ql2_star_L0) + qy_star_L10 * ql2_star_L0)
  Cl1g <- (dat$A == abar) / gA_pred

  qL1_fit <- SuperLearner(Y = dat$L1,
                          X = dat[, c("A", "W")],
                          family = binomial(),
                          SL.library = SL.library)
  hist_L <- dat
  hist_L$A <- abar
  offL1 <- predict(qL1_fit, type = "link", newdata = hist_L)$pred
  epsL1 <- glm(L1 ~ offset(offL1) + C_Q,
               weights = Cl1g,
               family = quasibinomial(),
               data = hist_L)$coef
  ql1_star <- plogis(offL1 + epsL1)

  ## ----- Final plug‑in estimate
  mean(
    qy_star_L11 * ql2_star_L1 * ql1_star +
    qy_star_L10 * ql2_star_L0 * (1 - ql1_star) +
    qy_star_L01 * (1 - ql2_star_L1) * ql1_star +
    qy_star_L00 * (1 - ql2_star_L0) * (1 - ql1_star)
  )
}

# ─────────────────────────────────────────────────────────────
# 2.  Run the 5 seeds  (2026 … 2030)
# ─────────────────────────────────────────────────────────────
seeds <- 2025 + 1:200
ori_out2 <- sapply(seeds, run_one_targeted_tmle_dgp2)

sim_results2 <- data.frame(seed = seeds, ori_estimate = ori_out2)
print(sim_results2, digits = 5)

bias2     <- mean(sim_results2$ori_estimate) - truth3
variance2 <- var(sim_results2$ori_estimate)
mse2      <- bias2^2 + variance2

metrics2 <- round(c(bias = bias2, variance = variance2, mse = mse2), 6)
print(metrics2)

In [ ]:
# ---- helper to run one Monte‑Carlo draw for DGP 2 (high censoring) ----
run_once_dgp2 <- function(seed) {
  set.seed(seed)

  n <- 500
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5 * W - 1))

  # time‑1 covariates
  L1 <- rbinom(n, 1, plogis(1.5 * W + 3 * A - W * A - 2))
  C1 <- rbinom(n, 1, 0.5)          # <-- higher censoring

  # time‑2 covariates
  L2 <- rbinom(n, 1, plogis(1.5 * W + 2 * A - 2 * W * A + L1 - 1))
  C2 <- rbinom(n, 1, 0.5)          # <-- higher censoring

  # observed values
  L2_obs <- ifelse(C1 == 1, NA, L2)

  # outcome (only if uncensored at both times)
  Y <- rbinom(
    n, 1,
    plogis(1.5 * W + 2 * A - 1.5 * W * A * L1 +
           3 * L1 - 1.5 * L1 * A - 2 - L2 * W + 2 * L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  # convert to “uncensored = 1” format required by ltmle
  C1 <- 1 - C1
  C2 <- 1 - C2

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)
  dat_clean <- na.omit(dat)  # keep fully observed rows

  SL.lib <- c("SL.glm.interaction", "SL.mean")

  # TMLE + IPTW
  fit_tmle <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib
  )

  # G‑computation
  fit_gcomp <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib,
    gcomp = TRUE
  )

  data.frame(
    seed  = seed,
    gcomp = as.numeric(fit_gcomp$estimates["gcomp"]),
    tmle  = as.numeric(fit_tmle$estimates["tmle"]),
    iptw  = as.numeric(fit_tmle$estimates["iptw"])
  )
}

# ---- run the simulation for seeds 2026 … 2030 ----
seeds <- 2025 + 1:200
sim_results_dgp2 <- do.call(rbind, lapply(seeds, run_once_dgp2))

# ---- bias, variance, MSE (uses the same truth3) ----
# make sure truth3 is already defined, e.g. truth3 <- 0.25
get_metrics <- function(x, truth) {
  bias <- mean(x) - truth
  varx <- var(x)
  mse  <- bias^2 + varx
  c(bias = bias, variance = varx, mse = mse)
}

metrics_dgp2 <- rbind(
  gcomp = get_metrics(sim_results_dgp2$gcomp, truth3),
  tmle  = get_metrics(sim_results_dgp2$tmle,  truth3),
  iptw  = get_metrics(sim_results_dgp2$iptw,  truth3)
)

print(round(metrics_dgp2, 6))


In [ ]:
# DGP 3 -informative
n <- 500

W <- rbinom(n, 1, 0.5)
A <- rbinom(n, 1, plogis(1.5*W - 1))

# Time 1 covariates
L1 <- rbinom(n, 1, plogis(1.5*W + 3*A - W*A - 2))
C1 <- rbinom(n, 1, plogis(-3+0.8*A+0.3*W+0.25*L1))

# Time 2 covariates (only for those not censored at time 1)
L2 <- rbinom(n, 1, plogis(1.5*W + 2*A - 2*W*A + L1 - 1))
C2 <- rbinom(n, 1, plogis(-3+0.8*A+0.3*W+0.25*L1+0.25*L2))
L2_obs <- ifelse(C1==1, NA, L2)

# Outcome Y only observed if C1 and C2 are both 0
Y <- rbinom(n, 1, plogis(
  1.5*W + 2*A - 1.5*W*A*L1 + 3*L1 - 1.5*L1*A - 2 - L2*W + 2*L2
))
Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

# Final data frame with correct temporal ordering
dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)



In [ ]:
# ─────────────────────────────────────────────────────────────
# 1.  One run under DGP 3 (informative censoring)
# ─────────────────────────────────────────────────────────────
run_one_targeted_tmle_dgp3 <- function(seed,
                                       n = 500,
                                       SL.library = c("SL.glm.interaction", "SL.mean"),
                                       abar = 1) {
  set.seed(seed)

  ## -------- DGP 3  (informative censoring) --------
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5 * W - 1))

  L1 <- rbinom(n, 1, plogis(1.5 * W + 3 * A - W * A - 2))
  C1 <- rbinom(n, 1, plogis(-3 + 0.8 * A + 0.3 * W + 0.25 * L1))

  L2 <- rbinom(n, 1, plogis(1.5 * W + 2 * A - 2 * W * A + L1 - 1))
  C2 <- rbinom(n, 1, plogis(-3 + 0.8 * A + 0.3 * W + 0.25 * L1 + 0.25 * L2))
  L2_obs <- ifelse(C1 == 1, NA, L2)

  Y <- rbinom(
    n, 1,
    plogis(1.5 * W + 2 * A - 1.5 * W * A * L1 +
           3 * L1 - 1.5 * L1 * A - 2 - L2 * W + 2 * L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)

  ## ---- identical targeting steps to previous functions ----
  # g‑models
  gA_fit  <- SuperLearner(Y = dat$A,
                          X = data.frame(W = dat$W),
                          family = binomial(),
                          SL.library = SL.library)
  gA_pred <- predict(gA_fit, newdata = data.frame(W = dat$W),
                     type = "response")$pred

  gC1_fit  <- SuperLearner(Y = dat$C1,
                           X = dat[, c("A", "W", "L1")],
                           family = binomial(),
                           SL.library = SL.library)
  gC1_pred <- predict(gC1_fit,
                      newdata = dat[, c("A", "W", "L1")],
                      type = "response")$pred

  idx_L2   <- !is.na(dat$L2_obs)
  gC2_fit  <- SuperLearner(Y = dat$C2[idx_L2],
                           X = dat[idx_L2, c("A", "W", "L1", "L2_obs")],
                           family = binomial(),
                           SL.library = SL.library)
  gC2_pred <- predict(gC2_fit,
                      newdata = dat[, c("A", "W", "L1", "L2_obs")],
                      type = "response")$pred

  # clever covariate for Y
  H_Y <- (dat$A == 1 & dat$C1 == 0 & dat$C2 == 0) /
         (gA_pred * (1 - gC1_pred) * (1 - gC2_pred))

  idx_Y  <- !is.na(dat$Y_obs)
  qY_fit <- SuperLearner(Y = dat$Y_obs[idx_Y],
                         X = dat[idx_Y, c("L2_obs", "L1", "A", "W")],
                         family = binomial(),
                         SL.library = SL.library)

  make_hist <- function(d, L1v, L2v) {
    h <- d; h$C1 <- h$C2 <- 0; h$A <- 1; h$L1 <- L1v; h$L2_obs <- L2v; h
  }
  update_Y <- function(hist) {
    off <- predict(qY_fit, type = "link", newdata = hist)$pred
    eps <- glm(Y_obs ~ offset(off) + 1,
               weights = H_Y,
               family  = quasibinomial(),
               data    = cbind(hist, Y_obs = dat$Y_obs))$coef
    plogis(off + eps)
  }
  qy_L11 <- update_Y(make_hist(dat, 1, 1))
  qy_L10 <- update_Y(make_hist(dat, 0, 1))
  qy_L01 <- update_Y(make_hist(dat, 1, 0))
  qy_L00 <- update_Y(make_hist(dat, 0, 0))

  # ---- Target L2
  Cl2q1 <- qy_L11 - qy_L01
  Cl2q0 <- qy_L10 - qy_L00
  Cl2g  <- (dat$A == abar & dat$C1 == 0) / (gA_pred * (1 - gC1_pred))

  qL2_fit <- SuperLearner(Y = dat$L2_obs[idx_L2],
                          X = dat[idx_L2, c("A", "W", "L1")],
                          family = binomial(),
                          SL.library = SL.library)

  hist_L1 <- make_hist(dat, 1, dat$L2_obs)
  off1 <- predict(qL2_fit, type = "link", newdata = hist_L1)$pred
  eps1 <- glm(L2_obs ~ offset(off1) + Cl2q1,
              weights = Cl2g,
              family  = quasibinomial(),
              data    = cbind(hist_L1, L2_obs = dat$L2_obs))$coef
  ql2_L1 <- plogis(off1 + eps1)

  hist_L0 <- make_hist(dat, 0, dat$L2_obs)
  off0 <- predict(qL2_fit, type = "link", newdata = hist_L0)$pred
  eps0 <- glm(L2_obs ~ offset(off0) + Cl2q0,
              weights = Cl2g,
              family  = quasibinomial(),
              data    = cbind(hist_L0, L2_obs = dat$L2_obs))$coef
  ql2_L0 <- plogis(off0 + eps0)

  # ---- Target L1
  C_Q <- qy_L01 * (1 - ql2_L1) + qy_L11 * ql2_L1 -
         (qy_L00 * (1 - ql2_L0) + qy_L10 * ql2_L0)
  Cl1g <- (dat$A == abar) / gA_pred

  qL1_fit <- SuperLearner(Y = dat$L1,
                          X = dat[, c("A", "W")],
                          family = binomial(),
                          SL.library = SL.library)
  hist_L <- dat; hist_L$A <- abar
  offL <- predict(qL1_fit, type = "link", newdata = hist_L)$pred
  epsL <- glm(L1 ~ offset(offL) + C_Q,
              weights = Cl1g,
              family  = quasibinomial(),
              data    = hist_L)$coef
  ql1_star <- plogis(offL + epsL)

  ## ---- Final plug‑in estimate
  mean(
    qy_L11 * ql2_L1 * ql1_star +
    qy_L10 * ql2_L0 * (1 - ql1_star) +
    qy_L01 * (1 - ql2_L1) * ql1_star +
    qy_L00 * (1 - ql2_L0) * (1 - ql1_star)
  )
}

# ─────────────────────────────────────────────────────────────
# 2.  Five seeds (2026 … 2030)
# ─────────────────────────────────────────────────────────────
seeds <- 2025 + 1:200
ori_out3 <- sapply(seeds, run_one_targeted_tmle_dgp3)

sim_results3 <- data.frame(seed = seeds, ori_estimate = ori_out3)
print(sim_results3, digits = 5)

# ─────────────────────────────────────────────────────────────
# 3.  Bias, variance, MSE  (truth3 is unchanged)
# ─────────────────────────────────────────────────────────────

bias3      <- mean(sim_results3$ori_estimate) - truth3
variance3  <- var(sim_results3$ori_estimate)
mse3       <- bias3^2 + variance3

metrics3 <- round(c(bias = bias3, variance = variance3, mse = mse3), 6)
print(metrics3)


In [ ]:
# ---- helper to run one Monte‑Carlo draw for DGP 3 ----
run_once_dgp3 <- function(seed) {
  set.seed(seed)

  n <- 500
  W <- rbinom(n, 1, 0.5)
  A <- rbinom(n, 1, plogis(1.5 * W - 1))

  # time‑1 covariates & informative censoring at t = 1
  L1 <- rbinom(n, 1, plogis(1.5 * W + 3 * A - W * A - 2))
  C1 <- rbinom(n, 1, plogis(-3 + 0.8 * A + 0.3 * W + 0.25 * L1))

  # time‑2 covariates & informative censoring at t = 2
  L2 <- rbinom(n, 1, plogis(1.5 * W + 2 * A - 2 * W * A + L1 - 1))
  C2 <- rbinom(n, 1, plogis(-3 + 0.8 * A + 0.3 * W + 0.25 * L1 + 0.25 * L2))

  # observed covariate at t = 2
  L2_obs <- ifelse(C1 == 1, NA, L2)

  # outcome, only observed if not censored at either time
  Y <- rbinom(
    n, 1,
    plogis(1.5 * W + 2 * A - 1.5 * W * A * L1 +
           3 * L1 - 1.5 * L1 * A - 2 - L2 * W + 2 * L2)
  )
  Y_obs <- ifelse(C1 == 1 | C2 == 1, NA, Y)

  # convert to “uncensored = 1” for ltmle
  C1 <- 1 - C1
  C2 <- 1 - C2

  dat <- data.frame(W, A, L1, C1, L2_obs, C2, Y_obs)
  dat_clean <- na.omit(dat)

  SL.lib <- c("SL.glm.interaction", "SL.mean")

  # TMLE + IPTW
  fit_tmle <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib
  )

  # G‑computation
  fit_gcomp <- ltmle(
    data   = dat_clean,
    Anodes = "A",
    Lnodes = c("L1", "L2_obs"),
    Cnodes = c("C1", "C2"),
    Ynodes = "Y_obs",
    abar   = 1,
    SL.library = SL.lib,
    gcomp = TRUE
  )

  data.frame(
    seed  = seed,
    gcomp = as.numeric(fit_gcomp$estimates["gcomp"]),
    tmle  = as.numeric(fit_tmle$estimates["tmle"]),
    iptw  = as.numeric(fit_tmle$estimates["iptw"])
  )
}

# ---- run the simulation for seeds 2026 … 2030 ----
seeds <- 2025 + 1:200
sim_results_dgp3 <- do.call(rbind, lapply(seeds, run_once_dgp3))

# ---- bias, variance, MSE relative to the same truth3 ----
# make sure truth3 is defined, e.g. truth3 <- 0.25
get_metrics <- function(x, truth) {
  bias <- mean(x) - truth
  varx <- var(x)
  mse  <- bias^2 + varx
  c(bias = bias, variance = varx, mse = mse)
}

metrics_dgp3 <- rbind(
  gcomp = get_metrics(sim_results_dgp3$gcomp, truth3),
  tmle  = get_metrics(sim_results_dgp3$tmle,  truth3),
  iptw  = get_metrics(sim_results_dgp3$iptw,  truth3)
)

print(round(metrics_dgp3, 6))
